# Notebook containing the sliding window approach for the Driving Paradigm

In [1]:
import numpy as np
import mne
import pickle
from mne.preprocessing import read_ica
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.svm import SVC, SVR
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from mne.decoding import CSP, SPoC
from continuous_control_bci.util import emg_classes_to_eeg_classes, SUBJECT_IDS
from tqdm import tqdm
import matplotlib as plt

from continuous_control_bci.data.load_data import load_calibration, load_driving
from continuous_control_bci.data.preprocessing import make_epochs, epochs_to_train_test
from continuous_control_bci.modelling.csp_classifier import create_csp_classifier, get_driving_epochs_for_csp
from sklearn.metrics import f1_score
from filterbank_csp import filter_bank, filter_bank_csp, predict_against_threshold, predict_against_threshold_indiv, create_csp_svm_classifier, majority_pred

In [13]:
# Subject 61 should have fairly good EEG data and performance
subject_id = "061"
include_rest = True
raw = load_calibration(subject_id)
raw.set_eeg_reference()
raw.filter(l_freq=5, h_freq=35)

ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
ica.apply(raw)

epochs = make_epochs(raw, include_rest=include_rest)

X_train, _, y_train, _ = epochs_to_train_test(epochs)
print("Training classifier. This may take a while..")
rank = {
    'eeg': X_train.shape[1] - len(ica.exclude),
    'mag': 32,
}
clf, y_pred = create_csp_classifier(X_train, y_train, rank)
svm_model, y_svm_pred = create_csp_svm_classifier(X_train, y_train, rank)
print("Classifier trained!")

Extracting EDF parameters from D:\RUG\Term 2\FYRP\data\ivo_data\sub-P061\motor-imagery-csp-061-acquisition-[2024.02.07-14.35.15].gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Channel 1, Channel 2, Channel 3, Channel 4, Channel 5, Channel 6, Channel 7, Channel 8, Channel 9, Channel 10, Channel 11, Channel 12, Channel 13, Channel 14, Channel 15, Channel 16, Channel 17, Channel 18, Channel 19, Channel 20, Channel 21, Channel 22, Channel 23, Channel 24, Channel 25, Channel 26, Channel 27, Channel 28, Channel 29, Channel 30, Channel 31, Channel 32, EX 1, EX 2, EX 3, EX 4, EX 5, EX 6, EX 7, EX 8
Creating raw.info structure...
Reading 0 ... 882783  =      0.000 ...   431.046 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter paramete

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading D:/RUG/Term 2/FYRP/data/ica/P061-calibration-ica.fif ...
Now restoring ICA solution ...


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.4s finished


Ready.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Used Annotations descriptions: ['769', '770']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['800']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Not setting metadata
80 matching events found
No baseline correction applied
Training classifier. This may take a while..

D:\RUG\Term 2\FYRP\FYRP\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)
D:\RUG\Term 2\FYRP\FYRP\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)



Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 3

In [14]:
# Loading the Driving data
driving = load_driving(subject_id)
driving.raw.set_eeg_reference()
driving.raw.filter(l_freq=5, h_freq=35)
ica.apply(driving.raw)

# EEG Data Stream
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
eeg_data, eeg_time = driving.raw[eeg_channel_indices]
# Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.2, preload=True, reject_by_annotation=False)

X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

# Making predictions based on EEG windows
# -1 left, 0 rest, 1 right
y_driving = driving.emg_prediction_stream['time_series']
y_driving = emg_classes_to_eeg_classes(y_driving)
y_driving = np.squeeze(y_driving)
print(X_driving.shape)
print(y_driving[:-1].shape)

Creating RawArray with float64 data, n_channels=40, n_times=5494682
    Range : 0 ... 5494681 =      0.000 ...  2682.950 secs
Ready.


D:\RUG\Term 2\FYRP\FYRP\continuous_control_bci\data\load_data.py:28: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  streams.raw.set_channel_types(CHANNEL_TYPE_MAPPING)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 3381 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    4.2s finished


Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Not setting metadata
13414 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13414 events and 410 original time points ...
0 bad epochs dropped
(13414, 32, 410)
(13414,)


In [15]:
y_preds, y_majority_pred = majority_pred(X_driving, clf, 10)
print(classification_report(y_driving[::10], y_majority_pred, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.52      0.95      0.67       444
       Right       0.61      0.56      0.58       290
        Rest       0.80      0.35      0.49       608

    accuracy                           0.59      1342
   macro avg       0.64      0.62      0.58      1342
weighted avg       0.67      0.59      0.57      1342



In [16]:
y_driving.shape

(13415,)

## 200ms windwos with 100 ms overlap

In [17]:
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
eeg_data, eeg_time = driving.raw[eeg_channel_indices]
# Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.3, overlap=0.1, preload=True, reject_by_annotation=False)

X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

Not setting metadata
13414 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13414 events and 614 original time points ...
0 bad epochs dropped


In [18]:
print(X_driving.shape)
print(y_driving.shape)

(13414, 32, 614)
(13415,)


In [19]:
# Am i doing this here correctly with the dimensions/ steps and then y_driving indexing?
y_preds_ol, y_majority_pred_ol = majority_pred(X_driving, clf, 10)
print(classification_report(y_driving[::10], y_majority_pred_ol, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.52      0.91      0.66       444
       Right       0.61      0.54      0.57       290
        Rest       0.73      0.37      0.49       608

    accuracy                           0.59      1342
   macro avg       0.62      0.61      0.58      1342
weighted avg       0.64      0.59      0.57      1342



# Now for all participants
    - 200ms Windwos, aggregated over 2seconds

In [11]:
#sub_ids = ["682", "854","968", "986"]

for subject_id in SUBJECT_IDS:
    # Subject 61 should have fairly good EEG data and performance
    include_rest = True
    raw = load_calibration(subject_id)
    raw.set_eeg_reference()
    raw.filter(l_freq=5, h_freq=35)

    ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
    ica.apply(raw)

    epochs = make_epochs(raw, include_rest=include_rest)

    X_train, _, y_train, _ = epochs_to_train_test(epochs)
    print("Training classifier. This may take a while..")
    rank = {
        'eeg': X_train.shape[1] - len(ica.exclude),
        'mag': 32,
    }
    clf, y_pred = create_csp_classifier(X_train, y_train, rank)
    svm_model, y_svm_pred = create_csp_svm_classifier(X_train, y_train, rank)
    print("Classifier trained!")

    # Loading the Driving data
    driving = load_driving(subject_id)
    driving.raw.set_eeg_reference()
    driving.raw.filter(l_freq=5, h_freq=35)
    ica.apply(driving.raw)

    # EEG Data Stream
    eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
    eeg_data, eeg_time = driving.raw[eeg_channel_indices]
    # Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
    eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.2, preload=True, reject_by_annotation=False)

    X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

    # EMG Stream
    # -1 left, 0 rest, 1 right
    y_driving = driving.emg_prediction_stream['time_series']
    y_driving = emg_classes_to_eeg_classes(y_driving)
    # 0 left, 1 right, 2 rest
    y_driving = np.squeeze(y_driving)
    print("Shape of y_driving before adjusting",y_driving.shape)
    print("Shape of X_driving before adjustment", X_driving.shape)
    if X_driving.shape[0] == y_driving.shape[0]:
        pass
    elif X_driving.shape[0] > y_driving.shape[0]:
        diff = X_driving.shape[0]-y_driving.shape[0]
        X_driving = X_driving[:-diff,:,:]
    else:
        diff = y_driving.shape[0]-X_driving.shape[0]
        y_driving = y_driving[:-diff]
    #print(X_driving.shape)
    #print(y_driving[:-1].shape)
    y_preds, y_majority_pred = majority_pred(X_driving, clf, 10)
    print("Shape of y-driving after adjustment",y_driving[::10].shape)
    print("Shape of Majority_pred",y_majority_pred.shape)
    report = classification_report(y_driving[::10], y_majority_pred, target_names=["Left", "Right", "Rest"], output_dict=True)
    print(report)
    with open(f'report_sliding_{subject_id}.pkl', 'wb') as fp:
        pickle.dump(report, fp)

Extracting EDF parameters from D:\RUG\Term 2\FYRP\data\ivo_data\sub-P682\motor-imagery-csp-682-acquisition-[2024.02.05-17.20.34].gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Channel 1, Channel 2, Channel 3, Channel 4, Channel 5, Channel 6, Channel 7, Channel 8, Channel 9, Channel 10, Channel 11, Channel 12, Channel 13, Channel 14, Channel 15, Channel 16, Channel 17, Channel 18, Channel 19, Channel 20, Channel 21, Channel 22, Channel 23, Channel 24, Channel 25, Channel 26, Channel 27, Channel 28, Channel 29, Channel 30, Channel 31, Channel 32, EX 1, EX 2, EX 3, EX 4, EX 5, EX 6, EX 7, EX 8
Creating raw.info structure...
Reading 0 ... 882687  =      0.000 ...   431.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter paramete

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading D:/RUG/Term 2/FYRP/data/ica/P682-calibration-ica.fif ...
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 3 ICA components
    Projecting back using 32 PCA components


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.4s finished


Used Annotations descriptions: ['769', '770']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['800']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Not setting metadata
80 matching events found
No baseline correction applied
Training classifier. This may take a while..


...

Computing rank from data with rank={'eeg': 28, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 28, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 28, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 28, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 28, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 28

D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\load_data.py:28: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  streams.raw.set_channel_types(CHANNEL_TYPE_MAPPING)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 3381 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.6s finished


Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 4 ICA components
    Projecting back using 32 PCA components
Not setting metadata
12967 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 12967 events and 410 original time points ...
0 bad epochs dropped
Shape of y_driving before adjusting (12970,)
Shape of X_driving before adjustment (12967, 32, 410)
Shape of y-driving after adjustment (1297,)
Shape of Majority_pred (1297,)
{'Left': {'precision': 0.3365617433414044, 'recall': 0.3797814207650273, 'f1-score': 0.3568677792041078, 'support': 366}, 'Right': {'precision': 0.3058823529411765, 'recall': 0.10970464135021098, 'f1-score': 0.1614906832298137, 'support': 237}, 'Rest': {'precision': 0.5757196495619524, 'recall': 0.6628242074927954, 'f1-score': 0.6162089752176826, 'support': 694}, 'accuracy': 0.4818812644564379, 'macro avg': {'precision': 0.4060545819481778, 'recall': 0.3841034

In [12]:
for subject_id in SUBJECT_IDS:
    with open(f"report_{subject_id}.pkl", "rb") as fp:
        report = pickle.load(fp)
        print(f"Subject {subject_id}")
        print("Left f1-score", report['Left']["f1-score"])
        print("Right f1-score", report['Right']["f1-score"])
        print("Rest f1-score", report['Rest']["f1-score"])
        print("Accuracy f1-score", report['accuracy'])
        print("\n")

Subject 061
Left f1-score 0.6741035856573706
Right f1-score 0.5827338129496402
Rest f1-score 0.4856815578465063
Accuracy f1-score 0.5938897168405365


Subject 066
Left f1-score 0.08178438661710037
Right f1-score 0.21929824561403508
Rest f1-score 0.636755823986195
Accuracy f1-score 0.4891304347826087


Subject 126
Left f1-score 0.20479999999999998
Right f1-score 0.4979919678714859
Rest f1-score 0.48270440251572333
Accuracy f1-score 0.42133131618759456


Subject 280
Left f1-score 0.44135188866799213
Right f1-score 0.330635838150289
Rest f1-score 0.24125452352231602
Accuracy f1-score 0.34444444444444444


Subject 302
Left f1-score 0.34805890227576974
Right f1-score 0.34151785714285715
Rest f1-score 0.36190476190476195
Accuracy f1-score 0.3508500772797527


Subject 336
Left f1-score 0.004866180048661801
Right f1-score 0.0
Rest f1-score 0.47155688622754494
Accuracy f1-score 0.3088954056695992


Subject 381
Left f1-score 0.6635601118359739
Right f1-score 0.6170212765957447
Rest f1-score 0.40

Seeing those results, they are all worse than the normal transfer learning but that was to be expected as this now is the harder task of finding when rest ends. The treshold is set to 0.2 because if the "hacked" version is used the threshold is 0 and then everything is predicted as rest for some reason.